In [55]:
import pandas as pd

# How to export CloudFit Azure RateCard

Run the kusto query:
```
CarbonEmissionResourceCard_Snapshot |
where Timestamp == '2022-08-01T00:00:00Z' |
join kind=inner MeterMapping_Materialized_Snapshot on MeterGuid |
summarize arg_max(Timestamp,*) by MeterGuid
```

Then export to CSV

In [56]:
az_co2_ratecard = pd.read_csv('./cloudfit/export.csv')

# CloudFit Azure RateCard

These units are units depending on the following classification

| Resource | Common Unit | Definition
|:--:|:--:|:--:|
| Storage | Gb-hour | One GB stored for one hour in Azure
| Compute | Core-hour | One core used for one hour based on the size of VM
| Network | GB | One GB of data transmitted 

In [57]:
az_co2_ratecard[az_co2_ratecard.MeterCategoryName == 'Virtual Machines']

,Group,MeterGuid,Timestamp,EmissionsPerUnitResourceUsage,EmissionsPerUnit_Scope1_mtCO2e_Azure,EmissionsPerUnit_Scope2_location_mtCO2e_Azure,EmissionsPerUnit_Scope2_market_mtCO2e_Azure,EmissionsPerUnit_Scope3_mtCo2e,MeterGuid1,MeterCategoryName,MeterSubcategoryName,MeterName,MeterRegionName
11516,NaN,039d49e6-c713-5929-a310-a9a53a018338,2022-08-01T00:00:00Z,2.583586e-04,3.404295e-07,4.922137e-06,0.000000,2.530960e-04,039d49e6-c713-5929-a310-a9a53a018338,Virtual Machines,Esv4 Series,E32s v4,SE Central
11517,NaN,04de6b3c-2dad-548b-9a78-5edf5cd19652,2022-08-01T00:00:00Z,4.120700e-05,5.487986e-08,2.382574e-05,0.000000,1.732637e-05,04de6b3c-2dad-548b-9a78-5edf5cd19652,Virtual Machines,NCasv3 T4 Series Windows,NC4as T4 v3,US Central
11518,NaN,058535f3-e927-4143-9b39-4e9ec2bcc825,2022-08-01T00:00:00Z,5.956743e-06,1.466834e-08,1.824460e-06,0.000000,4.117614e-06,058535f3-e927-4143-9b39-4e9ec2bcc825,Virtual Machines,D/DS Series,D11/DS11,US East
11519,NaN,081e60d3-d4d3-5b99-b839-587426650fd4,2022-08-01T00:00:00Z,7.196766e-05,9.656117e-07,2.242306e-05,0.000000,4.857899e-05,081e60d3-d4d3-5b99-b839-587426650fd4,Virtual Machines,Esv5 Series Windows,E48s v5 Low Priority,EU North
11520,NaN,08898009-bd5e-48c7-a926-1029e815fdc3,2022-08-01T00:00:00Z,1.242274e-06,5.704752e-09,6.344647e-07,0.000000,6.021049e-07,08898009-bd5e-48c7-a926-1029e815fdc3,Virtual Machines,Av2 Series,A2 v2 Low Priority,US South Central
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45325,NaN,ee6e2e65-b1d2-5863-a209-c584a58606a8,2022-08-01T00:00:00Z,3.418159e-06,3.901330e-09,1.182389e-06,0.000000,2.231868e-06,ee6e2e65-b1d2-5863-a209-c584a58606a8,Virtual Machines,Dsv4 Series,D2s v4,EU West
45326,NaN,f374ce73-912f-5a93-8fd2-d4a5986b4a52,2022-08-01T00:00:00Z,1.444544e-05,1.648736e-08,4.996881e-06,0.000000,9.432070e-06,f374ce73-912f-5a93-8fd2-d4a5986b4a52,Virtual Machines,Esv4 Series,E32-16s v4 Low Priority,EU West
45327,NaN,f69b71b2-4d12-406d-82f7-11903f972e6c,2022-08-01T00:00:00Z,1.706866e-05,3.556164e-08,8.988406e-06,0.000002,8.044690e-06,f69b71b2-4d12-406d-82f7-11903f972e6c,Virtual Machines,Av2 Series Windows,A4m v2,AU East
45328,NaN,fa752888-e8be-54d7-9723-4b44747350aa,2022-08-01T00:00:00Z,3.793761e-05,8.124693e-09,1.450433e-05,0.000000,2.342516e-05,fa752888-e8be-54d7-9723-4b44747350aa,Virtual Machines,D/DS Flex Series,D8/D8s,US West 3


# Constants

In [58]:
hrs_per_month = 730

# Functions to retrieve each component's (compute, network, storage) CO2 emissions

In [59]:
def get_emissions_for_vm_sku_in_kg(vm_sku, region='AU East'):
    return az_co2_ratecard[(az_co2_ratecard.MeterCategoryName == 'Virtual Machines') & (az_co2_ratecard.MeterName == vm_sku) & (az_co2_ratecard.MeterRegionName == region)].iloc[0].EmissionsPerUnitResourceUsage * hrs_per_month * 1000

In [60]:
def get_emissions_for_vm_disk_in_kg(disk_sku, region='AU East'):
    return az_co2_ratecard[(az_co2_ratecard.MeterCategoryName == 'Storage') & (az_co2_ratecard.MeterName == disk_sku) & (az_co2_ratecard.MeterRegionName == region)].iloc[0].EmissionsPerUnitResourceUsage * 1000

In [61]:
def get_emissions_for_vm_network(data_out_in_gb, region='All'):
    return (az_co2_ratecard[(az_co2_ratecard.MeterCategoryName == 'Virtual Network') & (az_co2_ratecard.MeterName == 'Inter-Region Egress') & (az_co2_ratecard.MeterRegionName == region)].iloc[0].EmissionsPerUnitResourceUsage + az_co2_ratecard[(az_co2_ratecard.MeterCategoryName == 'Virtual Network') & (az_co2_ratecard.MeterName == 'Inter-Region Ingress') & (az_co2_ratecard.MeterRegionName == region)].iloc[0].EmissionsPerUnitResourceUsage) * data_out_in_gb * 1000 

In [62]:
def get_emissions_for_vm_per_month(vm_sku, disk_sku, data_out_in_gb, region='AU East'):
    return get_emissions_for_vm_sku_in_kg(vm_sku, region) + get_emissions_for_vm_disk_in_kg(disk_sku, region) + get_emissions_for_vm_network(data_out_in_gb, 'All')

# Exercise functions

In [63]:
d2_v2_emissions =  get_emissions_for_vm_per_month('D2 v2/DS2 v2', 'S10 Disks', 50)
print(f'VM: {d2_v2_emissions} kg in CO2Eq Emissions/Month')

VM: 7.899807158759507 kg in CO2Eq Emissions/Month


In [64]:
d3_v2_emissions =  get_emissions_for_vm_per_month('D3 v2/DS3 v2', 'S10 Disks', 50)
print(f'VM: {d3_v2_emissions} kg in CO2Eq Emissions/Month')

VM: 15.302119696967235 kg in CO2Eq Emissions/Month


In [65]:
f2s_v2_emissions =  get_emissions_for_vm_per_month('F2s v2', 'S10 Disks', 50)
print(f'VM: {f2s_v2_emissions} kg in CO2Eq Emissions/Month')

VM: 5.845072717548771 kg in CO2Eq Emissions/Month


In [66]:
e2_v4_emissions =  get_emissions_for_vm_per_month('E2s v4', 'S10 Disks', 50)
print(f'VM: {e2_v4_emissions} kg in CO2Eq Emissions/Month')

VM: 6.898782687400419 kg in CO2Eq Emissions/Month


# AKS - Related Emissions

In [67]:
def get_emissions_for_aks_in_kg():
    return az_co2_ratecard[(az_co2_ratecard.MeterCategoryName == 'Azure Kubernetes Service') & (az_co2_ratecard.MeterName == 'Standard Uptime SLA')].iloc[0].EmissionsPerUnitResourceUsage * hrs_per_month * 1000

# Generate CloudFit Emissions for test data

## Azure VMs

In [68]:
vm_skus = ['E2s v4', 'F2s v2', 'D3 v2/DS3 v2', 'D2 v2/DS2 v2']
#regions = ['West US', 'East US', 'North Europe', 'West Europe', 'Southeast Asia', 'East Asia', 'Australia East', 'Australia Central']
# East Asia, Southeast Asia regions are missing
regions = ['US West', 'US East', 'EU North', 'EU West','AU East', 'AU Central']

In [69]:
# Compute emissions data for all regions and skus defined
emissions_data = []
for region in regions:
    for sku in vm_skus:
        emissions_data.append([region, sku, get_emissions_for_vm_per_month(sku, 'S10 Disks', 50, region)])

## Workload Units

We use the Workload Units to represent a single workload/app. For VMs, 1 Workload Unit = 1 VM instance.

In [70]:
emissions_df = pd.DataFrame(emissions_data, columns=['Region', 'VM SKU', 'Emissions per month(kg)'])
emissions_df['Workload Units = 3'] = emissions_df['Emissions per month(kg)'] * 3
emissions_df['Workload Units = 5'] = emissions_df['Emissions per month(kg)'] * 5

In [71]:
emissions_df

,Region,VM SKU,Emissions per month(kg),Workload Units = 3,Workload Units = 5
0,US West,E2s v4,4.926490,14.779471,24.632452
1,US West,F2s v2,4.251074,12.753221,21.255368
2,US West,D3 v2/DS3 v2,10.329825,30.989474,51.649124
3,US West,D2 v2/DS2 v2,3.098892,9.296676,15.494459
4,US East,E2s v4,5.250952,15.752856,26.254761
5,US East,F2s v2,2.245359,6.736077,11.226794
6,US East,D3 v2/DS3 v2,6.940753,20.822260,34.703767
7,US East,D2 v2/DS2 v2,6.016995,18.050986,30.084977
8,EU North,E2s v4,3.658375,10.975125,18.291875
9,EU North,F2s v2,2.601541,7.804623,13.007705


## AKS 

With AKS - we will introduce a density factor ```density_coeff```. This is effectively a multiplier which reduces the number of nodes required due to the assumed consolidation that will occur if we migrate VM workload to containers.

With AKS 1 Workload Unit = 1 VM instance * ```density_coeff```

In [72]:
density_coeff = 0.5

In [73]:
emissions_df['AKS Workload Units = 3'] = (emissions_df['Emissions per month(kg)'] * 3 * density_coeff) + get_emissions_for_aks_in_kg()
emissions_df['AKS Workload Units = 5'] = (emissions_df['Emissions per month(kg)'] * 5 * density_coeff) + get_emissions_for_aks_in_kg()

In [74]:
emissions_df

,Region,VM SKU,Emissions per month(kg),Workload Units = 3,Workload Units = 5,AKS Workload Units = 3,AKS Workload Units = 5
0,US West,E2s v4,4.926490,14.779471,24.632452,10.674934,15.601425
1,US West,F2s v2,4.251074,12.753221,21.255368,9.661809,13.912882
2,US West,D3 v2/DS3 v2,10.329825,30.989474,51.649124,18.779936,29.109761
3,US West,D2 v2/DS2 v2,3.098892,9.296676,15.494459,7.933537,11.032428
4,US East,E2s v4,5.250952,15.752856,26.254761,11.161627,16.412579
5,US East,F2s v2,2.245359,6.736077,11.226794,6.653237,8.898596
6,US East,D3 v2/DS3 v2,6.940753,20.822260,34.703767,13.696329,20.637082
7,US East,D2 v2/DS2 v2,6.016995,18.050986,30.084977,12.310692,18.327687
8,EU North,E2s v4,3.658375,10.975125,18.291875,8.772761,12.431136
9,EU North,F2s v2,2.601541,7.804623,13.007705,7.187510,9.789051
